In [1]:
# imports
import os
import csv

from bs4 import BeautifulSoup as bs

In [2]:
# dir
merged_csv_dir = r"D:\Debate_a_Base\Data\Parlamint_3.0\Csv_set\GB"
xml_dir = r"D:\Debate_a_Base\Data\Parlamint_3.0\Originele_taal\original_extract"

In [3]:
# vraagt alle files uit de folder op
def get_xml_files(country):
    os.chdir(xml_dir)
    
    country_return_list = []

    paths_dict = {}

    # ga door alle inhoud van de landfolder heen
    for root, dirs, files in os.walk(country):
        file_data = []

        # loop door files van een folder
        for file in files:

            #filter alleen de xml files
            if ".xml" in file and not "~" in file:
                file_data.append(file)

        # filter onzin uit de dict voor makkelijkere processing later
        if not "Schema" in root:
            paths_dict[root] = file_data
        
    return paths_dict

In [4]:
def get_xml_content_dict(path):
    os.chdir(xml_dir)
    
    content_dict = {}
    
    # lees het bestand
    with open(path, "r", encoding="utf-8") as file:
        # read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")
        
        # ga door alle segmenten info heen
        userlines = bs_content.find_all("u")
        for line in userlines:
    
            # definieer segment
            seg = line.get("xml:id")

            # definieer content (deze nieuwe manier negeert notes)
            mergable_lines = line.find_all("seg")
            cnt = ""
            for mergable_line in mergable_lines:

                cnt += f"{mergable_line.text} "
                
            content_dict[seg] = cnt
    
    return content_dict

In [5]:
# maak een csv waar segments zijn gemerged per file
def create_userline_csv(file, content):
    os.chdir(merged_csv_dir)
    file = file.replace(".xml", ".csv")
    
    with open(file, 'w', newline='', encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["key", "value"])
        
        for k, v in content.items():
            
            w.writerow([k, v])

In [6]:
# zet GB dataset om naar userline translation tier files
def scrap_gb(country):
    files = get_xml_files(country)
    
    # split hoofdfolder van mogelijke subfolders van jaren
    rootfolder = (k := next(iter(files)), files.pop(k))
    
    # ga door jaarfolders
    for year in files.keys():
        
        # ga elke file langs
        for file in files[year]:
            path = os.path.join(year, file)
            
            file_content_dict = get_xml_content_dict(path)
            
            create_userline_csv(file, file_content_dict)
        
    return True

In [7]:
scrap_this = 'ParlaMint-GB.TEI'

print(scrap_gb(scrap_this))

c:\users\asher\appdata\local\programs\python\python39\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


True
